In [133]:
import urllib
import xml.etree.cElementTree as etree
import pandas as pd

        
class FeedParser:
    def __init__(self, url):
        self.url = url        
        self.url_offer_df = self.read()
    def read(self):
        category = []
        vendor = []
        typePrefix = []
        model = []
        url = []
        #entire feed
        tree = etree.ElementTree(file="direct_moscow.php.xml")#etree.ElementTree(file=urllib.request.urlopen(url))#etree.ElementTree(file=urllib.request.urlopen('https://www.perfekto.ru/bitrix/catalog_export/direct_moscow.php'))#etree.ElementTree(reddit_data)
        #http://shop-sanequip.ru/market_shopsan_min.xml
        #http://budilkin.ru/feedsgr/vtochku.yml
        root = tree.getroot()
        
        categories = {}
        for cat in root.iter("category"):
            categories[cat.attrib["id"]] = cat.text
        counter = 0    
        for offer in root.iter("offer"):
            counter += 1
            if counter > 5:
                break
            if (offer.attrib["available"]=="true"):
                category.append(categories[offer.find("categoryId").text])       
                vendor.append(offer.find("vendor").text)
                url.append(offer.find("url").text)
                #print(offer.find("url").text)
                typePrefix.append(offer.find("typePrefix").text)
                try:
                    model.append(offer.find("model").text) 
                except AttributeError:
                    model.append(None)
        return(pd.DataFrame({"category":category, "vendor":vendor, "typePrefix":typePrefix, "model":model, 
                             "url":url}))

        
    def create_advertisements(self, templates):

        result = pd.DataFrame(columns=['num','model','keyword', 'header', 'url'])
        
        for template in templates:
            num = 1
            if (len(template) == 1):
                keyword = self.url_offer_df[template].loc[:,'model']
            else:
                keyword = self.url_offer_df[template].astype(str).apply(lambda x: " ".join(x),1)
            header = self.url_offer_df[["vendor","model"]].astype(str).apply(lambda x: "#" + " ".join(x) + "#",1)
            
            result = result.append(pd.DataFrame({"num": list(feed_parser.url_offer_df.index), 
                                   "model": self.url_offer_df.model, 
                                    "keyword":keyword,             
                                    "header": header, "url":self.url_offer_df.url}), ignore_index=True)
            num += 1
        result = result.sort_values("header")
        return(result)
    

        

In [134]:
feed_parser = FeedParser("")

In [135]:
templates = [["vendor","model"], ["category","model"], ["model"]]

adv = feed_parser.create_advertisements(templates)


['vendor', 'model']
['category', 'model']
['model']


In [136]:
adv

,header,keyword,model,num,url
0,#Aqualux PALERMO PALERMO 150x70#,Aqualux PALERMO PALERMO 150x70,PALERMO PALERMO 150x70,0,https://www.perfekto.ru/catalog/vanny/stalnaya...
3,#Aqualux PALERMO PALERMO 150x70#,Ванны PALERMO PALERMO 150x70,PALERMO PALERMO 150x70,0,https://www.perfekto.ru/catalog/vanny/stalnaya...
6,#Aqualux PALERMO PALERMO 150x70#,PALERMO PALERMO 150x70,PALERMO PALERMO 150x70,0,https://www.perfekto.ru/catalog/vanny/stalnaya...
1,#Aqualux Victory AQ-507AT#,Aqualux Victory AQ-507AT,Victory AQ-507AT,1,https://www.perfekto.ru/catalog/shower_room/du...
4,#Aqualux Victory AQ-507AT#,Душевые кабины и боксы Victory AQ-507AT,Victory AQ-507AT,1,https://www.perfekto.ru/catalog/shower_room/du...
7,#Aqualux Victory AQ-507AT#,Victory AQ-507AT,Victory AQ-507AT,1,https://www.perfekto.ru/catalog/shower_room/du...
2,#Medea А150L#,Medea А150L,А150L,2,https://www.perfekto.ru/catalog/vanny/vanna_le...
5,#Medea А150L#,Ванны А150L,А150L,2,https://www.perfekto.ru/catalog/vanny/vanna_le...
8,#Medea А150L#,А150L,А150L,2,https://www.perfekto.ru/catalog/vanny/vanna_le...
